## PUNTO 2

Desarrollar un crawler para el sitio web del Diario Jornada, que almacene la información de las secciones más relevantes de la página en una base de datos (puede usar Python + BeautifulSoup). Con la información recabada, resolver y graficar las siguientes operaciones.

- Mostrar cuáles son las 5 noticias más relevantes según cantidad de visitas a nivel global y por sección.
- Mostrar cómo están distribuidas las palabras de cierto artı́culo.
- Mostrar el número promedio, máximo y mı́nimo de palabras, sentencias, párrafos de un conjunto de noticias de su preferencia.

In [1]:
from lxml import html
import requests
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sqlalchemy import create_engine
import sqlite3
import pandas as pd
import os

In [2]:
def get_noticia(url):
    noticia = {}
    page = requests.get(url)
    response = html.fromstring(page.content)
    noticia['categoria'] = response.xpath('//a[@id="ContentPlaceHolder1_hl_Seccion"]/text()',namespaces={"re": "http://exslt.org/regular-expressions"})
    noticia['titulo'] = response.xpath('//span[@id="ContentPlaceHolder1_lbl_Titulo"]/text()',namespaces={"re": "http://exslt.org/regular-expressions"})
    noticia['subtitulo'] = response.xpath('//span[@id="ContentPlaceHolder1_lbl_Copete"]/text()',namespaces={"re": "http://exslt.org/regular-expressions"})
    noticia['cuerpo'] = response.xpath('//div[@id="cuerpo"]/p/text()',namespaces={"re": "http://exslt.org/regular-expressions"})
    noticia['cant_leidas'] = response.xpath('//span[@id="ContentPlaceHolder1_lbl_Hits"]/text()',namespaces={"re": "http://exslt.org/regular-expressions"})
    return noticia

In [3]:
def get_noticias_from_seccion(url_seccion):
    page = requests.get(url_seccion)
    response = html.fromstring(page.content)
    noticias_importantes_str = '//div[re:test(@id, "ContentPlaceHolder1_home_bloque_seccion_RP_Bloque_\d_pnl_Titulo_\d$")]//a/@href'
    noticias_secundarias_str = '//a[re:test(@id, "ContentPlaceHolder1_RP_Noticias_hl_Titulo_\d$")]/@href'
    url_noticias_1 = response.xpath(noticias_importantes_str,namespaces={"re": "http://exslt.org/regular-expressions"})
    url_noticias_2 = response.xpath(noticias_secundarias_str,namespaces={"re": "http://exslt.org/regular-expressions"})
    return url_noticias_1 + url_noticias_2

In [4]:
url_ppal = "https://www.diariojornada.com.ar/"
categorias = ["provincia","policiales","sociedad","deportes"]
noticias = {}
for categoria in categorias:
    url_noticias = get_noticias_from_seccion(url_ppal+categoria)
    noticias[categoria] = [get_noticia(url) for url in url_noticias]    

### Almacenamiento en una BD

Para este caso particular guardo los datos en una base SQLite. El modulo sqlite3 viene incorporado con python3 por lo que sirve como almacenamiento de las noticias obtenidas por el crawler.

In [5]:
#Cargamos los datos en un DataFrame
df = pd.DataFrame()
for k,v in noticias.items():
    c = pd.DataFrame(v)
    df = df.append(c)
df = df.applymap(lambda row: ''.join(row))

In [6]:
#Cargamos el DataFrame en la base de datos sqlite
here = os.path.abspath('')
engine = create_engine('sqlite:///'+here+'/db_crawler.db')
sqlite_connection = engine.connect()
sqlite_table = "crawler_diario_jornada"
df.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

### Mostrar cuáles son las 5 noticias más relevantes según cantidad de visitas a nivel global y por sección.

In [ ]:
for categoria, lista_noticias in noticias.items():
    top5 = sorted(lista_noticias, key = lambda i: i['cant_leidas'],reverse=True)[:5] 
    print(categoria.upper())
    for noticia in top5:
        print('Titulo: '+noticia['titulo'][0])
        print('Cant. Leidas: '+noticia['cant_leidas'][0])
    print("..............................")

### Mostrar cómo están distribuidas las palabras de cierto artı́culo.

In [ ]:
for categoria, lista_noticias in noticias.items():
    top5 = sorted(lista_noticias, key = lambda i: i['cant_leidas'],reverse=True)[:5] 
    print(categoria.upper())
    for noticia in top5:
        print('Titulo: '+noticia['titulo'][0])
        print("Recuento de palabras:")
        r= sorted(Counter(" ".join(noticia['cuerpo']).split(" ")).items(), key=lambda item: item[1], reverse=True)[:20]
        x = dict(r)
        keys = x.keys()
        vals = x.values()
        plt.bar(keys, np.divide(list(vals), sum(vals)), label="Recuento de palabras por noticia")
        plt.ylim(0,1)
        plt.ylabel ('Percentage')
        plt.xlabel ('Significant number')
        plt.xticks(list(keys), rotation=60)
        plt.legend (bbox_to_anchor=(1, 1), loc="upper right", borderaxespad=0.)
        plt.show()

### Mostrar el número promedio, máximo y mı́nimo de palabras, sentencias, párrafos de un conjunto de noticias de su preferencia.

In [ ]:
r = []
for k,v in noticias.items():
    resultado = {}
    l_palabras=[]
    l_parrafos=[]
    for noticia in v:
        l_palabras.append(len(" ".join(noticia['cuerpo']).split(" ")))
        l_parrafos.append(len(" ".join(noticia['cuerpo']).split(".")))
    resultado['categoria'] = k
    resultado['palabras'] = l_palabras
    resultado['parrafos'] = l_parrafos  
    r.append(resultado)

In [ ]:
for noticia in r:
    print("###################################################################")
    print("----------------- Categoria: "+ noticia['categoria'] +"------------")
    print("-------------------------------------------------------------------")
    print(F"Cantidad maxima de palabras en las noticias de la categoria: Provincia fueron {max(noticia['palabras'])}")
    print(F"Cantidad minima de palabras en las noticias de la categoria: Provincia fueron {min(noticia['palabras'])}")
    print(F'Promedio de palabras por noticia: {round(np.average(l_palabras))}')
    print("-------------------------------------------------------------------")
    print(F"Cantidad Maxima de parrafos en las noticias de la categoria: Provincia fueron {max(noticia['parrafos'])} ")
    print(F"Cantidad Maxima de parrafos en las noticias de la categoria: Provincia fueron {min(noticia['parrafos'])} ")
    print(F'Promedio de parrafos por noticia: {round(np.average(l_parrafos))}')